# Proof of Work

Gráfica de dificultad [Blockchain.com](https://www.blockchain.com/es/explorer/charts/difficulty).

Recálculo de dificultad cada 2016 bloques.

In [3]:
import hashlib
import time

## Cálculo de PoW

Adaptado de Andreas Antonopoulos, *Mastering Bitcoin*.

In [7]:
max_nonce = 2 ** 32  # 4 billion
max_nonce

4294967296

In [8]:



def proof_of_work(header, difficulty_bits) -> tuple[str, int] | tuple[None, None]:
    # calculate the difficulty target
    target = 2 ** (256 - difficulty_bits)

    for nonce in range(max_nonce):
        hash_result = hashlib.sha256(
            (str(header) + str(nonce)).encode("utf-8")
        ).hexdigest()

        # check if this is a valid result, below the target
        if int(hash_result, 16) < target:
            print(f"Success with nonce {nonce}")
            print(f"Hash is {hash_result}")
            return (hash_result, nonce)

    print(f"Failed after {max_nonce} tries")
    return None, None

In [9]:
nonce = 0
hash_result = ""

# difficulty from 0 to 31 bits
for difficulty_bits in range(32):
    difficulty = 2 ** difficulty_bits
    print(f"\nDifficulty: {difficulty} ({difficulty_bits} bits)")
    print("Starting search...")

    # checkpoint the current time
    start_time = time.time()

    # make a new block which includes the hash from the previous block
    # we fake a block of transactions - just a string
    new_block = "test block with transactions" + hash_result

    # find a valid nonce for the new block
    hash_result, nonce = proof_of_work(new_block, difficulty_bits)

    # checkpoint how long it took to find a result
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Elapsed Time: {elapsed_time:.4f} seconds")

    if elapsed_time > 0 and nonce is not None:
        # estimate the hashes per second
        hash_power = float(nonce / elapsed_time)
        print(f"Hashing Power: {int(hash_power)} hashes per second")


Difficulty: 1 (0 bits)
Starting search...
Success with nonce 0
Hash is ff8253ed10b5f719d52a709a66af8cd5e2054f702e675af4ca0cae70f0988634
Elapsed Time: 0.0008 seconds
Hashing Power: 0 hashes per second

Difficulty: 2 (1 bits)
Starting search...
Success with nonce 0
Hash is 22c608547e239faf5c353e7ebd204042760b93891d1d0be9ab488d36c73c077b
Elapsed Time: 0.0000 seconds
Hashing Power: 0 hashes per second

Difficulty: 4 (2 bits)
Starting search...
Success with nonce 2
Hash is 0635f41cdb98c6e73516f84fc88da19a13a3bac6298dbfc0df5170bac93ba4dd
Elapsed Time: 0.0000 seconds
Hashing Power: 155344 hashes per second

Difficulty: 8 (3 bits)
Starting search...
Success with nonce 9
Hash is 1c1c105e65b47142f028a8f93ddf3dabb9260491bc64474738133ce5256cb3c1
Elapsed Time: 0.0000 seconds
Hashing Power: 629145 hashes per second

Difficulty: 16 (4 bits)
Starting search...
Success with nonce 25
Hash is 0f7becfd3bcd1a82e06663c97176add89e7cae0268de46f94e7e11bc3863e148
Elapsed Time: 0.0000 seconds
Hashing Power: 103

## PoW Real

In [11]:
from dataclasses import dataclass

# -------------------------
# Utilidades de serialización y hashing
# -------------------------
def u32_le(n: int) -> bytes:
    """Entero de 32 bits a little-endian (4 bytes)."""
    return n.to_bytes(4, byteorder="little", signed=False)

def hex_le(s: str) -> bytes:
    """
    Toma un hex mostrado en "formato humano" (big-endian),
    y lo convierte a bytes little-endian como exige el header.
    """
    b = bytes.fromhex(s)
    return b[::-1]  # invertir a little-endian

def dbl_sha256(b: bytes) -> bytes:
    """double-SHA256"""
    return hashlib.sha256(hashlib.sha256(b).digest()).digest()

# -------------------------
# Conversión nBits (compact) -> target entero
# -------------------------
def bits_to_target(bits: int) -> int:
    """
    nBits es un formato compacto: 1 byte de exponente y 3 de mantisa.
    target = mantissa * 256^(exponent-3)
    """
    exponent = bits >> 24
    mantissa = bits & 0xFFFFFF
    return mantissa * (1 << (8 * (exponent - 3)))

def hash_to_int_be(h: bytes) -> int:
    """Interpreta el hash (bytes) como entero big-endian."""
    return int.from_bytes(h, byteorder="big")

# -------------------------
# Estructura del header de bloque
# -------------------------
@dataclass
class BlockHeader:
    version: int          # 4 bytes LE
    prev_block: str       # 32 bytes (hex big-endian de cara al usuario)
    merkle_root: str      # 32 bytes (hex big-endian de cara al usuario)
    timestamp: int        # 4 bytes LE (UNIX epoch)
    bits: int             # 4 bytes LE (compact)
    nonce: int            # 4 bytes LE

    def serialize(self) -> bytes:
        """
        Serializa el header exactamente como lo hace Bitcoin:
        - version, time, bits, nonce en little-endian
        - prev_block y merkle_root como hashes little-endian (hex invertido)
        """
        return (
            u32_le(self.version) +
            hex_le(self.prev_block) +
            hex_le(self.merkle_root) +
            u32_le(self.timestamp) +
            u32_le(self.bits) +
            u32_le(self.nonce)
        )

    def pow_hash(self) -> bytes:
        """double-SHA256 del header serializado."""
        return dbl_sha256(self.serialize())

    def is_valid_pow(self) -> bool:
        target = bits_to_target(self.bits)
        h = self.pow_hash()
        return hash_to_int_be(h) <= target

In [12]:
# -------------------------
# 1) Verificar PoW del bloque génesis
#    Datos públicos conocidos
# -------------------------
genesis = BlockHeader(
    version=1,
    prev_block="00"*32,  # 32 bytes de cero
    merkle_root="4a5e1e4baab89f3a32518a88c31bc87f618f76673e2cc77ab2127b7afdeda33b",
    timestamp=1231006505,   # 2009-01-03 18:15:05 UTC
    bits=0x1d00ffff,        # dificultad 1 (target inicial)
    nonce=2083236893
)

g_hash = genesis.pow_hash()               # bytes (big-endian al imprimir .hex())
g_hash_hex_be = g_hash.hex()              # representación humana
g_hash_hex_display = g_hash[::-1].hex()   # cómo se muestra típicamente en exploradores (LE->BE)

print("=== GENESIS BLOCK ===")
print("Header hash (big-endian):   ", g_hash_hex_be)
print("Hash mostrado (convención): ", g_hash_hex_display)
print("Target (desde nBits):       ", hex(bits_to_target(genesis.bits)))
print("¿PoW válido?:               ", genesis.is_valid_pow())

=== GENESIS BLOCK ===
Header hash (big-endian):    6fe28c0ab6f1b372c1a6a246ae63f74f931e8365e15a089c68d6190000000000
Hash mostrado (convención):  000000000019d6689c085ae165831e934ff763ae46a2a6c172b3f1b60a8ce26f
Target (desde nBits):        0xffff0000000000000000000000000000000000000000000000000000
¿PoW válido?:                False


In [21]:
2<<8

512

In [18]:
# -------------------------
# 2) Mini “minado” educativo
#    Buscamos un nonce para un target MUY fácil (no real de Bitcoin),
#    para que encuentre solución rápido con pocos intentos.
# -------------------------
easy_header = BlockHeader(
    version=1,
    prev_block="11"*32,
    merkle_root="22"*32,
    timestamp=1700000000,
    bits=0x997fffff,  # target extremadamente fácil (exponente 0x20, mantisa ~0x7fffff)
    nonce=0
)

easy_target = bits_to_target(easy_header.bits)
print("\n=== MINERÍA EDUCATIVA (target fácil) ===")
print("Target fácil: ", hex(easy_target))

found = None
for nonce in range(0, 5_000_000):
    easy_header.nonce = nonce
    h = easy_header.pow_hash()
    if hash_to_int_be(h) <= easy_target:
        found = (nonce, h)
        break

if found:
    nonce, h = found
    print(f"¡Encontrado! nonce={nonce}")
    print("Hash:", h.hex())
    print("Hash (estilo explorador):", h[::-1].hex())
else:
    print("No se encontró un nonce en el rango probado (sube el rango o facilita más el target).")


=== MINERÍA EDUCATIVA (target fácil) ===
Target fácil:  0x7fffff000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000
¡Encontrado! nonce=0
Hash: 45d329b23282d73b5e94eabf4b1fe9650b6fda983c1381e93481fa145d69ba2f
Hash (estilo explorador): 2fba695d14fa8134e981133c98da6f0b65e91f4bbfea945e3bd78232b229d345
